# Waymo Driveless Car Data Analysis Project 
## Converting image sequence to video

Team 2: Yumeng Jiang, Gerardo Antonio Lopez Ruiz, **Qian Zheng**

Date: Sep 29th 2019

## 0 Sample Video

https://youtu.be/3L6d292qmC8

## 1 Prerequisite Installation

In [0]:
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.2.3)
     |████████████████████████████████| 2.7MB 4.9MB/s 


## 2 Essentials Importing

In [0]:
import tensorflow as tf
import cv2

tf.enable_eager_execution()

from waymo_open_dataset import dataset_pb2 as open_dataset

## 3 Images Loading

Each segment contains several frames, and each frame contains five images taken by 5 camera at different positions (front, front left, side left, front right and side right). Given that the video this task generates serves for determining the scenario of each segment, we only use the front-view image of each frame (POS == 0).

In [0]:
FILENAME = '/content/segment-169115044301335945_480_000_500_000.tfrecord'

dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')

# Load frames from dataset
frames = []
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    frames.append(frame)
    
print("filename:", FILENAME, "Num of frames:", len(frames))

# ["FRONT", "FRONT_LEFT", "SIDE_LEFT", "FRONT_RIGHT", "SIDE_RIGHT"]
POS = 0

# Read img from waymo img class into numpy arrays
imgs = []
for frame in frames:
  l = list(enumerate(frame.images))
  _, im = l[POS]
  img = tf.image.decode_jpeg(im.image)
  height, width, _ = img.shape
  size = (width,height)
  imgs.append(img.numpy())

filename: /content/segment-169115044301335945_480_000_500_000.tfrecord Num of frames:  198


## 4 Conversion to Video

FPS refers to frames per second. 24 is used by most of films.

In [0]:
output = FILENAME.split('.')[0] + ".mp4"
fps = 24

# Write a sequence of imgs to a video  
out = cv2.VideoWriter(output, cv2.VideoWriter_fourcc('M','J','P','G'), fps, size)
for img in imgs:
    out.write(img)
out.release()